# Package Imports

In [1]:
import pandas as pd
import numpy as np
import cmd
import time

start_time = time.time()
cli = cmd.Cmd()

# Helper Functions

In [2]:
## create a function which will assign a decade for each season
## goal of this is to represent voting patterns across different eras
def assign_decade(df, season_col='Season'):
    df['decade'] = ( (df[season_col] // 10) * 10).astype(str) + 's'
    return df

# Rookie Pitchers

In [3]:
### Relief Pitchers have at least 40 IP

relief = pd.read_csv('../data/rookies/fg_relief.csv')
relief.drop(columns=['NameASCII', 'PlayerId'], inplace=True)
relief = relief[~(relief['Season'].isin( [1994, 2020] ))] # exclude 1994 and 2020 shortened seasons

relief_columns = relief.columns.tolist()
print(f"Columns for Relief Pitchers: \n")
print(cli.columnize(relief_columns, displaywidth=50))

Columns for Relief Pitchers: 

Season  SO    E-F    K%+     Clutch  WAR    
Name    ERA   K/BB+  BB%+    SV      MLBAMID
Team    ERA-  HR/9+  K-BB%   BS    
G       FIP   AVG+   WPA     SD    
IP      FIP-  WHIP+  WPA/LI  MD    
None


In [4]:
relief['BS'] = relief['BS'].fillna(0) # fill in blown saves with 0 (might not have existed yet)

# Add decade columns for pitchers
relief = assign_decade(relief)

## Add a 'WAR/IP' for a secondary representation of WAR
## This can be better for comparing people's true WAR impact across relievers and starters
relief['WAR/IP'] = relief['WAR'] / relief['IP']

## Add 'IP/G' to help distinguish between relief pitchers and closing pitchers.
relief['IP/G'] = relief['IP'] / relief['G']

## Add 'SD_pct' to evaluate how much more frequently they are throwing a Shutdown
## Shutdown (SD) = Relief Appearance with cumulative WPA > 0.6 (good)
## Meltdown (MD) = Relief Appearance with cumulative WPA < -0.6 (bad)
relief['SD_pct'] = relief['SD'] / (relief['SD'] + relief['MD'])

relief['pos'] = 'relief' # create the relief pitcher designation

## Closer conditions
cond1 = ( ( (relief['SV'] + relief['BS']) / relief['G'] ) >= 0.4)
cond2 = (relief['IP/G'] <= 1.25)

# Distinguish between a relief pitcher, and a closer (yes, they are different)
relief.loc[cond1 & cond2 , 'pos'] = 'closer' 

In [5]:
### Starting Pitchers have at least 100 IP
starter = pd.read_csv('../data/rookies/fg_starter.csv')
starter.drop(columns=['NameASCII', 'PlayerId'], inplace=True)
starter = starter[~(starter['Season'].isin( [1994, 2020] ))]

starter_columns = starter.columns.tolist()
print(f"Columns for Starting Pitchers:")
print(cli.columnize(starter_columns, displaywidth=50))

Columns for Starting Pitchers:
Season  G    IP    FIP-   WHIP+  WPA/LI 
Name    GS   SO    E-F    K%+    Clutch 
Team    CG   ERA   K/BB+  BB%+   WAR    
W       ShO  ERA-  HR/9+  K-BB%  MLBAMID
L       QS   FIP   AVG+   WPA  
None


In [6]:
starter = assign_decade(starter)

starter['WAR/IP'] = starter['WAR'] / starter['IP']

starter['pos'] = 'starter' # create the starting pitcher designation

## Checking for Missing Values

The missing data in both the pitcher data is represented by a `NaN` values. These values are meaningful in this context for the statistics missing data: `BS`, `QS`, `Barrel%`, and `HardHit%`. This is because these statistics are newer and the data simply does not exist.

In [7]:
print("Missing Values in Relief Pitchers Data")
print(relief.isna().sum())

Missing Values in Relief Pitchers Data
Season     0
Name       0
Team       0
G          0
IP         0
SO         0
ERA        0
ERA-       0
FIP        0
FIP-       0
E-F        0
K/BB+      0
HR/9+      0
AVG+       0
WHIP+      0
K%+        0
BB%+       0
K-BB%      0
WPA        0
WPA/LI     0
Clutch     0
SV         0
BS         0
SD         0
MD         0
WAR        0
MLBAMID    0
decade     0
WAR/IP     0
IP/G       0
SD_pct     0
pos        0
dtype: int64


In [8]:
print("Missing Values in Starter Pitchers Data")
print(starter.isna().sum())

Missing Values in Starter Pitchers Data
Season     0
Name       0
Team       0
W          0
L          0
G          0
GS         0
CG         0
ShO        0
QS         0
IP         0
SO         0
ERA        0
ERA-       0
FIP        0
FIP-       0
E-F        0
K/BB+      0
HR/9+      0
AVG+       0
WHIP+      0
K%+        0
BB%+       0
K-BB%      0
WPA        0
WPA/LI     0
Clutch     0
WAR        0
MLBAMID    0
decade     0
WAR/IP     0
pos        0
dtype: int64


In [9]:
print("The unique teams in the Dataframe are: \n")
print(f"Number of Teams = {len(relief['Team'].unique())} \n")
print(relief['Team'].unique()) # Find the unique team occurrences

The unique teams in the Dataframe are: 

Number of Teams = 36 

['TOR' 'LAD' 'NYY' 'BOS' 'ATL' 'SEA' 'MIN' 'STL' 'MIL' 'CAL' 'CLE' 'OAK'
 'HOU' 'MIA' 'KCR' 'CIN' '- - -' 'SDP' 'TEX' 'BAL' 'DET' 'CHC' 'PHI' 'PIT'
 'CHW' 'SFG' 'LAA' 'NYM' 'ANA' 'TBR' 'FLA' 'MON' 'ARI' 'COL' 'WSN' 'TBD']


### Table of Teams (by League)

| NL Abbreviation | NL Team Name            | AL Abbreviation | AL Team Name            |
|-----------------|-------------------------|-----------------|-------------------------|
| ARI             | Arizona Diamondbacks     | BAL             | Baltimore Orioles       |
| ATL             | Atlanta Braves           | BOS             | Boston Red Sox          |
| CHC             | Chicago Cubs             | CHW             | Chicago White Sox       |
| CIN             | Cincinnati Reds          | CLE             | Cleveland Guardians     |
| COL             | Colorado Rockies         | DET             | Detroit Tigers          |
| LAD             | Los Angeles Dodgers      | HOU             | Houston Astros          |
| MIA             | Miami Marlins            | KCR             | Kansas City Royals      |
| MIL             | Milwaukee Brewers        | LAA             | Los Angeles Angels      |
| NYM             | New York Mets            | MIN             | Minnesota Twins         |
| PHI             | Philadelphia Phillies    | NYY             | New York Yankees        |
| PIT             | Pittsburgh Pirates       | OAK             | Oakland Athletics       |
| SDP             | San Diego Padres         | SEA             | Seattle Mariners        |
| SFG             | San Francisco Giants     | TBR             | Tampa Bay Rays          |
| STL             | St. Louis Cardinals      | TEX             | Texas Rangers           |
| WSN             | Washington Nationals     | TOR             | Toronto Blue Jays       |


In [10]:
# Dictionary for mapping old abbreviations to current ones
team_map = {
    'CAL': 'LAA',  # California Angels to Los Angeles Angels
    'ANA': 'LAA',  # Anaheim Angels to Los Angeles Angels
    'FLA': 'MIA',  # Florida Marlins to Miami Marlins
    'MON': 'WSN',  # Montreal Expos to Washington Nationals
    'TBD': 'TBR'   # Tampa Bay Devil Rays to Tampa Bay Rays
}

# Replace the 'Team' column with updated abbreviations using .replace()
relief['Team'] = relief['Team'].replace(team_map)
starter['Team'] = starter['Team'].replace(team_map)

In [11]:
# National League
nl_teams = ['LAD', 'SDP', 'CIN', 'SFG', 'ATL',
            'NYM', 'MIA', 'MIL', 'CHC', 'STL',
            'COL', 'PIT', 'ARI', 'PHI', 'WSN']
# American League
al_teams = ['OAK', 'HOU', 'LAA', 'SEA', 'CLE',
            'BAL', 'NYY', 'BOS', 'KCR', 'MIN',
            'TOR', 'TEX', 'CHW', 'TBR', 'DET']

# add league for the Relievers
relief.loc[relief['Team'].isin(nl_teams), 'league'] = 'NL'
relief.loc[relief['Team'].isin(al_teams), 'league'] = 'AL'
# add league for the Starters
starter.loc[starter['Team'].isin(nl_teams), 'league'] = 'NL'
starter.loc[starter['Team'].isin(al_teams), 'league'] = 'AL'

In [12]:
print(f"There are {starter.shape[0]} Starting Pitchers in the data")
print(f"There are {relief.shape[0]} Relief Pitchers in the data")

There are 482 Starting Pitchers in the data
There are 762 Relief Pitchers in the data


## Rookie of the Year

In [13]:
## Load Rookie of the Year data
roy = pd.read_csv('../data/awards/roy_awards.csv')
roy.rename(columns={'player_id' : 'rookie_of_the_year'}, inplace=True) # rename the column for merging

print(f"Columns for Rookie of the Year data:")
print(roy.columns.tolist())
print("\n")

Columns for Rookie of the Year data:
['award_id', 'season', 'rookie_of_the_year']




In [14]:
## Merge the relief and starter data with the Rookie of the Year data
relief = pd.merge(relief, roy,
        left_on = 'Season', right_on = 'season',
        how = 'left')

relief.drop(columns='season', inplace=True) # unnecessary column

# Rename the award_id values
relief.loc[relief['award_id'] == 'ALROY', 'award_id'] = 'AL'
relief.loc[relief['award_id'] == 'NLROY', 'award_id'] = 'NL'

relief.dropna(subset='league', inplace=True)

starter = pd.merge(starter, roy,
        left_on = 'Season', right_on = 'season',
        how = 'left')

starter.drop(columns='season', inplace=True) # unnecessary column

# Rename the award_id values
starter.loc[starter['award_id'] == 'ALROY', 'award_id'] = 'AL'
starter.loc[starter['award_id'] == 'NLROY', 'award_id'] = 'NL'

starter.dropna(subset='league', inplace=True)

In [15]:
## Separate the data for 2024 from the rest of the data
## The 2024 data will be used to make predictions after the model creation process

relief_24 = relief[relief['Season'] == 2024]
starter_24 = starter[starter['Season'] == 2024]

relief = relief[relief['Season'] != 2024]
starter = starter[starter['Season'] != 2024]

In [16]:
relief = relief[relief['league'] == relief['award_id']]
starter = starter[starter['league'] == starter['award_id']]

## Create an indicator column for those who have won rookie of the year

relief['rookie_of_the_year'] = np.where(relief['rookie_of_the_year'] == relief['MLBAMID'], 1, 0)
starter['rookie_of_the_year'] = np.where(starter['rookie_of_the_year'] == starter['MLBAMID'], 1, 0)

print(relief.head(10))
print("\n")
print(starter.head(10))

    Season               Name Team   G     IP   SO       ERA       ERA-  \
0     1986      Mark Eichhorn  TOR  69  157.0  166  1.719745  40.860984   
3     2006      Takashi Saito  LAD  72   78.1  107  2.068086  46.986878   
4     1985       Brian Fisher  NYY  55   98.1   85  2.379661  58.890406   
6     2006  Jonathan Papelbon  BOS  59   68.1   75  0.921952  19.523037   
8     2014    Dellin Betances  NYY  70   90.0  135  1.400000  36.423515   
11    2011      Craig Kimbrel  ATL  79   77.0  127  2.103896  55.827688   
13    1982    Steve Bedrosian  ATL  61  122.0  114  2.286885  60.687104   
14    1977       Enrique Romo  SEA  55   98.2   92  3.010135  72.789408   
16    1980       Doug Corbett  MIN  73  136.1   89  1.980440  47.446845   
19    2016       Seunghwan Oh  STL  76   79.2  103  1.920503  47.095538   

         FIP       FIP-  ...       WAR  MLBAMID  decade    WAR/IP      IP/G  \
0   2.312413  56.211928  ...  4.944612   113780   1980s  0.031494  2.275362   
3   1.870152  41

## Create Rank Columns

In order to accurately compare players across positions with differing statistical comparisons, as there are with both pitchers and batters I'm deciding to summarize their respective skill across various categories by ranking them using the following:

- `mean_rookie_rank`: The mean rank of all the categories ranked
- `rookie_rank`: The overall rank amongst all rookies (The ranking of `mean_rookie_rank`)
- `mean_mlb_rank`: The mean rank of all the categories ranked (for all MLB players)
- `mlb_rank`: The overall rank amongst all MLB Players (The ranking of `mean_mlb_rank`)

In [17]:
cli.columnize(relief.columns.tolist(), displaywidth=50)

Season  ERA-   WHIP+   SV       WAR/IP            
Name    FIP    K%+     BS       IP/G              
Team    FIP-   BB%+    SD       SD_pct            
G       E-F    K-BB%   MD       pos               
IP      K/BB+  WPA     WAR      league            
SO      HR/9+  WPA/LI  MLBAMID  award_id          
ERA     AVG+   Clutch  decade   rookie_of_the_year


In [18]:
## Lower value indicates better performance
low_cols = ['ERA-', 'FIP-', 'HR/9+', 'WHIP+', 
            'AVG+', 'BB%+', 'MD']

## higher value indicates better performance
high_cols = ['WAR', 'WAR/IP', 'IP', 'K%+', 'WPA', 
             'WPA/LI', 'K/BB+', 'K-BB%',
             'SD', 'SD_pct', 'Clutch']

In [19]:
# Rank the low_cols in descending order with percentile-based ranking
relief[[col + '_rank' for col in low_cols]] = relief.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                      na_option='keep',
                                                                                      pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
relief[[col + '_rank' for col in high_cols]] = relief.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                        na_option='keep',
                                                                                        pct=True)

# Select columns that end with '_rank'
rank_columns = relief.filter(regex='_rank$').columns.tolist()

# row means for rank columns
relief['mean_rookie_rank'] = relief[rank_columns].mean(axis=1)

# Rank mean_rookie_rank in descending order
relief['rookie_rank'] = relief.groupby('Season')['mean_rookie_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [20]:
# Rank the low_cols in descending order with percentile-based ranking
relief_24[[col + '_rank' for col in low_cols]] = relief_24.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                      na_option='keep',
                                                                                      pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
relief_24[[col + '_rank' for col in high_cols]] = relief_24.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                              na_option='keep', 
                                                                                              pct=True)

# Select columns that end with '_rank'
rank_columns = relief_24.filter(regex='_rank$').columns.tolist()

relief_24['mean_rookie_rank'] = relief_24[rank_columns].mean(axis=1)

# Rank mean_rookie_rank in descending order 
relief_24['rookie_rank'] = relief_24.groupby('Season')['mean_rookie_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [21]:
add_cols = ['rookie_of_the_year', 'mean_rookie_rank', 'rookie_rank']
rank_columns = rank_columns + add_cols

rank_columns.insert(0, 'Name')
rank_columns.insert(1, 'Season')
rank_columns.insert(2, 'pos')

In [22]:
print(relief.loc[:, rank_columns].sort_values('mean_rookie_rank', ascending=False).head(10))

                 Name  Season     pos  ERA-_rank  FIP-_rank  HR/9+_rank  \
24      Troy Percival    1995  relief   1.000000   1.000000    0.857143   
8     Dellin Betances    2014  relief   0.933333   0.933333    0.600000   
119       Jhoan Duran    2022  relief   0.966667   0.966667    0.633333   
83      Huston Street    2005  closer   1.000000   1.000000    0.863636   
42       Joakim Soria    2007  relief   0.850000   1.000000    0.700000   
0       Mark Eichhorn    1986  relief   1.000000   1.000000    0.857143   
124  Kerry Ligtenberg    1998  closer   0.900000   1.000000    0.700000   
82          Ken Giles    2014  relief   1.000000   1.000000    1.000000   
58        Gregg Olson    1989  relief   1.000000   1.000000    1.000000   
159      Tim Crabtree    1996  relief   1.000000   1.000000    1.000000   

     WHIP+_rank  AVG+_rank  BB%+_rank   MD_rank  ...  WPA_rank  WPA/LI_rank  \
24     1.000000   1.000000   0.857143  0.928571  ...  1.000000     1.000000   
8      1.000000 

In [23]:
print(relief_24.loc[:, rank_columns].sort_values('mean_rookie_rank', ascending=False).head(10))

                Name  Season     pos  ERA-_rank  FIP-_rank  HR/9+_rank  \
26        Cade Smith    2024  relief   0.909091   1.000000    0.954545   
31      Mason Miller    2024  closer   0.772727   0.954545    0.681818   
141    Hunter Gaddis    2024  relief   1.000000   0.727273    0.772727   
418     Bryan Hudson    2024  relief   0.954545   0.590909    0.227273   
171  Orion Kerkering    2024  relief   0.863636   0.909091    0.863636   
242    Justin Slaten    2024  relief   0.636364   0.772727    0.545455   
217  Justin Martinez    2024  relief   0.818182   0.818182    0.909091   
189    Declan Cronin    2024  relief   0.181818   0.863636    1.000000   
626   Ryan Fernandez    2024  relief   0.590909   0.500000    0.727273   
762   Tyler Ferguson    2024  relief   0.136364   0.409091    0.363636   

     WHIP+_rank  AVG+_rank  BB%+_rank   MD_rank  ...  WPA_rank  WPA/LI_rank  \
26     0.863636   0.727273   0.772727  0.613636  ...  0.863636     0.954545   
31     0.909091   0.954545 

In [24]:
print(cli.columnize(starter.columns.tolist(), displaywidth=50))

Season  CG    FIP    K%+     MLBAMID           
Name    ShO   FIP-   BB%+    decade            
Team    QS    E-F    K-BB%   WAR/IP            
W       IP    K/BB+  WPA     pos               
L       SO    HR/9+  WPA/LI  league            
G       ERA   AVG+   Clutch  award_id          
GS      ERA-  WHIP+  WAR     rookie_of_the_year
None


In [25]:
low_cols = ['ERA-', 'FIP-', 'HR/9+', 'WHIP+', 
            'AVG+', 'BB%+']

high_cols = ['WAR', 'WAR/IP', 'IP', 'K%+', 'WPA', 
             'WPA/LI', 'K/BB+', 'K-BB%', 'Clutch', 'QS']

In [26]:
# Rank the low_cols in descending order with percentile-based ranking
starter[[col + '_rank' for col in low_cols]] = starter.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                        na_option='keep',
                                                                                        pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
starter[[col + '_rank' for col in high_cols]] = starter.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                          na_option='keep',
                                                                                          pct=True)

# Select columns that end with '_rank'
rank_columns = starter.filter(regex='_rank$').columns.tolist()

starter['mean_rookie_rank'] = starter[rank_columns].mean(axis=1)

# Rank mean_rookie_rank in descending order with percentile-based ranking
starter['rookie_rank'] = starter.groupby('Season')['mean_rookie_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [27]:
# Rank the low_cols in descending order with percentile-based ranking
starter_24[[col + '_rank' for col in low_cols]] = starter_24.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                              na_option='keep', 
                                                                                              pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
starter_24[[col + '_rank' for col in high_cols]] = starter_24.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                                na_option='keep',
                                                                                                pct=True)

# Select columns that end with '_rank'
rank_columns = starter_24.filter(regex='_rank$').columns.tolist()

starter_24['mean_rookie_rank'] = starter_24[rank_columns].mean(axis=1)

# Rank mean_rookie_rank in descending order with percentile-based ranking
starter_24['rookie_rank'] = starter_24.groupby('Season')['mean_rookie_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [28]:
add_cols = ['rookie_of_the_year', 'mean_rookie_rank', 'rookie_rank']
rank_columns = rank_columns + add_cols

rank_columns.insert(0, 'Name')
rank_columns.insert(1, 'Season')
rank_columns.insert(2, 'pos')

In [29]:
print(starter.loc[:, rank_columns].sort_values('mean_rookie_rank', ascending=False).head(10))

                    Name  Season      pos  ERA-_rank  FIP-_rank  HR/9+_rank  \
22          Brandon Webb    2003  starter   1.000000   1.000000    1.000000   
3        John Montefusco    1975  starter   0.833333   1.000000    1.000000   
280          Bill Laskey    1982  starter   1.000000   1.000000    0.666667   
15           Britt Burns    1980  starter   1.000000   0.857143    0.571429   
5             Hideo Nomo    1995  starter   1.000000   1.000000    0.769231   
230         John Fulgham    1979  starter   1.000000   0.916667    0.833333   
7    Fernando Valenzuela    1981  starter   0.800000   0.800000    0.400000   
46       Edinson Volquez    2008  starter   1.000000   1.000000    0.909091   
36        Michael Soroka    2019  starter   1.000000   1.000000    1.000000   
1          Dwight Gooden    1984  starter   0.916667   1.000000    1.000000   

     WHIP+_rank  AVG+_rank  BB%+_rank  WAR_rank  ...  K%+_rank  WPA_rank  \
22     1.000000   1.000000   0.416667       1.0  ...  

In [30]:
print(starter_24.loc[:, rank_columns].sort_values('mean_rookie_rank', ascending=False).head(10))

                        Name  Season      pos  ERA-_rank  FIP-_rank  \
63               Paul Skenes    2024  starter   1.000000   1.000000   
203            Shota Imanaga    2024  starter   0.833333   0.750000   
245                 Luis Gil    2024  starter   0.750000   0.916667   
361              Gavin Stone    2024  starter   0.666667   0.666667   
501              Jared Jones    2024  starter   0.583333   0.583333   
525             Tobias Myers    2024  starter   0.916667   0.416667   
334          Mitchell Parker    2024  starter   0.416667   0.833333   
433  Simeon Woods Richardson    2024  starter   0.500000   0.500000   
635               Joey Estes    2024  starter   0.166667   0.083333   
540       Spencer Arrighetti    2024  starter   0.083333   0.333333   

     HR/9+_rank  WHIP+_rank  AVG+_rank  BB%+_rank  WAR_rank  ...  K%+_rank  \
63     1.000000    1.000000   0.916667   0.750000  1.000000  ...  1.000000   
203    0.083333    0.916667   0.750000   1.000000  0.916667  .

In [31]:
rookie_relief = relief
rookie_starter = starter

rookie_relief_24 = relief_24
rookie_starter_24 = starter_24

# All of MLB Pitchers

In [32]:
relief = pd.read_csv('../data/mlb/fg_relief.csv')
relief.drop(columns=['NameASCII', 'PlayerId'], inplace=True)
relief = relief[~(relief['Season'].isin( [1994, 2020] ))]

relief_columns = relief.columns.tolist()
print(f"Columns for Relief Pitchers: \n")
print(cli.columnize(relief_columns, displaywidth=50))

Columns for Relief Pitchers: 

Season  SO    E-F    K%+     Clutch  WAR    
Name    ERA   K/BB+  BB%+    SV      MLBAMID
Team    ERA-  HR/9+  K-BB%   BS    
G       FIP   AVG+   WPA     SD    
IP      FIP-  WHIP+  WPA/LI  MD    
None


In [33]:
relief['BS'] = relief['BS'].fillna(0)

# Add decade columns for pitchers
relief = assign_decade(relief)

## Add a 'WAR/IP' for a secondary representation of WAR
## This can be better for comparing people's true WAR impact across relievers and starters
relief['WAR/IP'] = relief['WAR'] / relief['IP']

## Add 'IP/G' to help distinguish between relief pitchers and closing pitchers.
relief['IP/G'] = relief['IP'] / relief['G']

## Add 'SD_pct' to evaluate how much more frequently they are throwing a Shutdown
## Shutdown (SD) = Relief Appearance with cumulative WPA > 0.6 (good)
## Meltdown (MD) = Relief Appearance with cumulative WPA < -0.6 (bad)
relief['SD_pct'] = relief['SD'] / (relief['SD'] + relief['MD'])

relief['pos'] = 'relief' # create the relief pitcher designation
cond1 = ( ( (relief['SV'] + relief['BS']) / relief['G'] ) >= 0.4)
cond2 = (relief['IP/G'] <= 1.25)

# Distinguish between a relief pitcher, and a closer (yes, they are different)
relief.loc[cond1 & cond2 , 'pos'] = 'closer' 

In [34]:
starter = pd.read_csv('../data/mlb/fg_starter.csv')
starter.drop(columns=['NameASCII', 'PlayerId'], inplace=True)
starter = starter[~(starter['Season'].isin( [1994, 2020] ))]

starter_columns = starter.columns.tolist()
print(f"Columns for Starting Pitchers:")
print(cli.columnize(starter_columns, displaywidth=50))

Columns for Starting Pitchers:
Season  G    IP    FIP-   WHIP+  WPA/LI 
Name    GS   SO    E-F    K%+    Clutch 
Team    CG   ERA   K/BB+  BB%+   WAR    
W       ShO  ERA-  HR/9+  K-BB%  MLBAMID
L       QS   FIP   AVG+   WPA  
None


In [35]:
starter = assign_decade(starter)

starter['WAR/IP'] = starter['WAR'] / starter['IP']

starter['pos'] = 'starter' # create the starting pitcher designation

In [36]:
# Dictionary for mapping old abbreviations to current ones
team_map = {
    'CAL': 'LAA',  # California Angels to Los Angeles Angels
    'ANA': 'LAA',  # Anaheim Angels to Los Angeles Angels
    'FLA': 'MIA',  # Florida Marlins to Miami Marlins
    'MON': 'WSN',  # Montreal Expos to Washington Nationals
    'TBD': 'TBR'   # Tampa Bay Devil Rays to Tampa Bay Rays
}

# Replace the 'Team' column with updated abbreviations using .replace()
relief['Team'] = relief['Team'].replace(team_map)
starter['Team'] = starter['Team'].replace(team_map)

In [37]:
# National League
nl_teams = ['LAD', 'SDP', 'CIN', 'SFG', 'ATL',
            'NYM', 'MIA', 'MIL', 'CHC', 'STL',
            'COL', 'PIT', 'ARI', 'PHI', 'WSN']
# American League
al_teams = ['OAK', 'HOU', 'LAA', 'SEA', 'CLE',
            'BAL', 'NYY', 'BOS', 'KCR', 'MIN',
            'TOR', 'TEX', 'CHW', 'TBR', 'DET']

# add league for the Relievers
relief.loc[relief['Team'].isin(nl_teams), 'league'] = 'NL'
relief.loc[relief['Team'].isin(al_teams), 'league'] = 'AL'
# add league for the Starters
starter.loc[starter['Team'].isin(nl_teams), 'league'] = 'NL'
starter.loc[starter['Team'].isin(al_teams), 'league'] = 'AL'

relief.dropna(subset='league', inplace=True)
starter.dropna(subset='league', inplace=True)

In [38]:
print(f"There are {starter.shape[0]} Starting Pitchers in the data")
print(f"There are {relief.shape[0]} Relief Pitchers in the data")

There are 5518 Starting Pitchers in the data
There are 6356 Relief Pitchers in the data


In [39]:
relief_24 = relief[relief['Season'] == 2024]
starter_24 = starter[starter['Season'] == 2024]

relief = relief[relief['Season'] != 2024]
starter = starter[starter['Season'] != 2024]

In [40]:
print(relief.head(10))
print("\n")
print(starter.head(10))
print("\n")
print(relief_24.head(10))
print("\n")
print(starter_24.head(10))

   Season            Name Team    G     IP   SO       ERA       ERA-  \
0    1977    Bruce Sutter  CHC   62  107.1  129  1.341615  31.241640   
1    1986   Mark Eichhorn  TOR   69  157.0  166  1.719745  40.860984   
2    1979    Bruce Sutter  CHC   62  101.1  110  2.220395  54.203137   
3    1979        Jim Kern  TEX   71  143.0  136  1.573427  37.783756   
4    2003      Eric Gagne  LAD   77   82.1  137  1.202429  29.796020   
5    1990      Rob Dibble  CIN   68   98.0  136  1.744898  44.466424   
6    1996  Mariano Rivera  NYY   61  107.2  130  2.089783  42.868730   
7    1977    Rich Gossage  PIT   72  133.0  151  1.624060  41.020796   
8    1974   Mike Marshall  LAD  106  208.1  143  2.419200  70.285680   
9    1976  Rollie Fingers  OAK   70  134.2  113  2.539604  74.156263   

        FIP       FIP-  ...  SD  MD       WAR  MLBAMID  decade    WAR/IP  \
0  1.610714  38.518270  ...  39  10  5.157516   123004   1970s  0.048156   
1  2.312413  56.211928  ...  32  10  4.944612   113780 

In [41]:
low_cols = ['ERA-', 'FIP-', 'HR/9+', 'WHIP+', 
            'AVG+', 'BB%+', 'MD']

high_cols = ['WAR', 'WAR/IP', 'IP', 'K%+', 'WPA', 
             'WPA/LI', 'K/BB+', 'K-BB%',
             'SD', 'SD_pct', 'Clutch']

In [42]:
# Rank the low_cols in descending order with percentile-based ranking
relief[[col + '_rank' for col in low_cols]] = relief.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                      na_option='keep',
                                                                                      pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
relief[[col + '_rank' for col in high_cols]] = relief.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                        na_option='keep',
                                                                                        pct=True)

# Select columns that end with '_rank'
rank_columns = relief.filter(regex='_rank$').columns.tolist()

relief['mean_mlb_rank'] = relief[rank_columns].mean(axis=1)

# Rank mean_mlb_rank in descending order with percentile-based ranking
relief['mlb_rank'] = relief.groupby('Season')['mean_mlb_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [43]:
# Rank the low_cols in descending order with percentile-based ranking
relief_24[[col + '_rank' for col in low_cols]] = relief_24.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                      na_option='keep',
                                                                                      pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
relief_24[[col + '_rank' for col in high_cols]] = relief_24.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                              na_option='keep', 
                                                                                              pct=True)

# Select columns that end with '_rank'
rank_columns = relief_24.filter(regex='_rank$').columns.tolist()

relief_24['mean_mlb_rank'] = relief_24[rank_columns].mean(axis=1)

# Rank mean_mlb_rank in descending order with percentile-based ranking
relief_24['mlb_rank'] = relief_24.groupby('Season')['mean_mlb_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [44]:
add_cols = ['mean_mlb_rank', 'mlb_rank']
rank_columns = rank_columns + add_cols

rank_columns.insert(0, 'Name')
rank_columns.insert(1, 'Season')
rank_columns.insert(2, 'pos')

In [45]:
print(relief.loc[:, rank_columns].sort_values('mean_mlb_rank', ascending=False).head(10))

                 Name  Season     pos  ERA-_rank  FIP-_rank  HR/9+_rank  \
28      Kenley Jansen    2017  closer   0.987097   1.000000    0.716129   
37   Willie Hernandez    1984  relief   0.981308   0.981308    0.869159   
46     Trevor Hoffman    1998  closer   0.986014   0.993007    0.958042   
40   Dennis Eckersley    1990  closer   1.000000   1.000000    0.933884   
4          Eric Gagne    2003  closer   0.993056   1.000000    0.958333   
20      Blake Treinen    2018  closer   1.000000   0.987179    0.980769   
57         Joe Nathan    2006  closer   0.972789   1.000000    0.925170   
6      Mariano Rivera    1996  relief   0.984615   1.000000    1.000000   
298         J.J. Putz    2007  closer   0.993789   0.937888    0.490683   
17         Eric Gagne    2002  closer   0.887324   0.992958    0.598592   

     WHIP+_rank  AVG+_rank  BB%+_rank   MD_rank  ...  K%+_rank  WPA_rank  \
28     0.987097   0.922581   1.000000  0.967742  ...  0.993548  1.000000   
37     0.990654   0.96

In [46]:
print(relief_24.loc[:, rank_columns].sort_values('mean_mlb_rank', ascending=False).head(10))

                Name  Season     pos  ERA-_rank  FIP-_rank  HR/9+_rank  \
316   Emmanuel Clase    2024  closer   1.000000   0.972414    0.951724   
546  Raisel Iglesias    2024  closer   0.993103   0.944828    0.931034   
192       Cade Smith    2024  relief   0.910345   1.000000    0.979310   
540      Ryan Walker    2024  relief   0.937931   0.924138    0.875862   
207     Mason Miller    2024  closer   0.868966   0.986207    0.813793   
331     Jeff Hoffman    2024  relief   0.958621   0.937931    0.675862   
206      Griffin Jax    2024  relief   0.931034   0.993103    0.903448   
497      Kirby Yates    2024  closer   0.986207   0.931034    0.882759   
867    Hunter Gaddis    2024  relief   0.979310   0.813793    0.855172   
388     Ryan Helsley    2024  closer   0.889655   0.875862    0.896552   

     WHIP+_rank  AVG+_rank  BB%+_rank   MD_rank  ...  K%+_rank  WPA_rank  \
316    0.993103   0.951724   0.979310  0.920690  ...  0.517241  1.000000   
546    1.000000   0.986207   0.91

In [47]:
print(cli.columnize(starter.columns.tolist(), displaywidth=50))

Season  G    IP    FIP-   WHIP+  WPA/LI   WAR/IP
Name    GS   SO    E-F    K%+    Clutch   pos   
Team    CG   ERA   K/BB+  BB%+   WAR      league
W       ShO  ERA-  HR/9+  K-BB%  MLBAMID
L       QS   FIP   AVG+   WPA    decade 
None


In [48]:
low_cols = ['ERA-', 'FIP-', 'HR/9+', 'WHIP+', 
            'AVG+', 'BB%+']

high_cols = ['WAR', 'WAR/IP', 'IP', 'K%+', 'WPA', 
             'WPA/LI', 'K/BB+', 'K-BB%', 'Clutch', 'QS']

In [49]:
# Rank the low_cols in descending order with percentile-based ranking
starter[[col + '_rank' for col in low_cols]] = starter.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                        na_option='keep',
                                                                                        pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
starter[[col + '_rank' for col in high_cols]] = starter.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                          na_option='keep',
                                                                                          pct=True)

# Select columns that end with '_rank'
rank_columns = starter.filter(regex='_rank$').columns.tolist()

starter['mean_mlb_rank'] = starter[rank_columns].mean(axis=1)

# Rank mean_mlb_rank in descending order with percentile-based ranking
starter['mlb_rank'] = starter.groupby('Season')['mean_mlb_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [50]:
# Rank the low_cols in descending order with percentile-based ranking
starter_24[[col + '_rank' for col in low_cols]] = starter_24.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                              na_option='keep', 
                                                                                              pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
starter_24[[col + '_rank' for col in high_cols]] = starter_24.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                                na_option='keep',
                                                                                                pct=True)

# Select columns that end with '_rank'
rank_columns = starter_24.filter(regex='_rank$').columns.tolist()

starter_24['mean_mlb_rank'] = starter_24[rank_columns].mean(axis=1)

# Rank mean_mlb_rank in descending order with percentile-based ranking
starter_24['mlb_rank'] = starter_24.groupby('Season')['mean_mlb_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [51]:
add_cols = ['mean_mlb_rank', 'mlb_rank']
rank_columns = rank_columns + add_cols

rank_columns.insert(0, 'Name')
rank_columns.insert(1, 'Season')
rank_columns.insert(2, 'pos')

In [52]:
print(starter.loc[:, rank_columns].sort_values('mean_mlb_rank', ascending=False).head(10))

               Name  Season      pos  ERA-_rank  FIP-_rank  HR/9+_rank  \
14    Dwight Gooden    1985  starter   1.000000   1.000000    0.933333   
36      Greg Maddux    1995  starter   1.000000   0.989474    0.989474   
31    Roger Clemens    1990  starter   1.000000   1.000000    1.000000   
17     Zack Greinke    2009  starter   1.000000   1.000000    0.981818   
12     Jacob deGrom    2018  starter   1.000000   0.991525    0.991525   
44  Bret Saberhagen    1989  starter   1.000000   1.000000    0.951923   
77     Jake Arrieta    2015  starter   0.991870   0.991870    1.000000   
1     Roger Clemens    1997  starter   1.000000   1.000000    0.990741   
97      Kevin Brown    2000  starter   0.974359   0.982906    0.871795   
6     Randy Johnson    1995  starter   0.989474   1.000000    0.968421   

    WHIP+_rank  AVG+_rank  BB%+_rank  WAR_rank  ...   IP_rank  K%+_rank  \
14    0.990476   0.990476   0.771429  1.000000  ...  1.000000  0.990476   
36    1.000000   0.989474   1.00000

In [53]:
print(starter_24.loc[:, rank_columns].sort_values('mean_mlb_rank', ascending=False).head(10))

                Name  Season      pos  ERA-_rank  FIP-_rank  HR/9+_rank  \
330     Tarik Skubal    2024  starter   0.959184   0.989796    0.897959   
160       Chris Sale    2024  starter   0.979592   1.000000    1.000000   
537     Zack Wheeler    2024  starter   0.948980   0.887755    0.785714   
1242     Paul Skenes    2024  starter   0.989796   0.979592    0.857143   
900    Logan Gilbert    2024  starter   0.724490   0.877551    0.724490   
718        Seth Lugo    2024  starter   0.928571   0.846939    0.918367   
600      Cole Ragans    2024  starter   0.806122   0.938776    0.846939   
1778  Reynaldo López    2024  starter   1.000000   0.897959    0.887755   
1165   Hunter Greene    2024  starter   0.938776   0.836735    0.908163   
1430  Framber Valdez    2024  starter   0.897959   0.867347    0.938776   

      WHIP+_rank  AVG+_rank  BB%+_rank  WAR_rank  ...   IP_rank  K%+_rank  \
330     0.959184   0.877551   0.918367  0.989796  ...  0.918367  0.959184   
160     0.938776   0

In [54]:
mlb_relief = relief.copy()
mlb_relief_24 = relief_24.copy()
mlb_starter = starter.copy()
mlb_starter_24 = starter_24.copy()

In [55]:
rookie_relief = pd.merge(rookie_relief, mlb_relief[['Season', 'MLBAMID', 'mlb_rank', 'mean_mlb_rank']],
                           on = ['Season', 'MLBAMID'],
                           how='left'
                          ).sort_values('mean_mlb_rank', 
                                        ascending=False)

rookie_relief['rookie_rank'] = rookie_relief.groupby('Season')['mean_rookie_rank'].rank(ascending=False, 
                                                                                        na_option='keep').astype('int64')

rookie_relief_24 = pd.merge(rookie_relief_24, mlb_relief_24[['Season', 'MLBAMID', 'mlb_rank', 'mean_mlb_rank']],
                           on = ['Season', 'MLBAMID'],
                           how='left'
                          ).sort_values('mean_mlb_rank', 
                                        ascending=False)

rookie_relief_24['rookie_rank'] = rookie_relief_24.groupby('Season')['mean_rookie_rank'].rank(ascending=False, 
                                                                                              na_option='keep').astype('int64')

In [56]:
rookie_starter = pd.merge(rookie_starter, mlb_starter[['Season', 'MLBAMID', 'mlb_rank', 'mean_mlb_rank']],
                           on = ['Season', 'MLBAMID'],
                           how='left'
                          ).sort_values('mean_mlb_rank', 
                                        ascending=False)

rookie_starter['rookie_rank'] = rookie_starter.groupby('Season')['mean_rookie_rank'].rank(ascending=False, 
                                                                                        na_option='keep').astype('int64')

rookie_starter_24 = pd.merge(rookie_starter_24, mlb_starter_24[['Season', 'MLBAMID', 'mlb_rank', 'mean_mlb_rank']],
                           on = ['Season', 'MLBAMID'],
                           how='left'
                          ).sort_values('mean_mlb_rank', 
                                        ascending=False)

rookie_starter_24['rookie_rank'] = rookie_starter_24.groupby('Season')['mean_rookie_rank'].rank(ascending=False, 
                                                                                                na_option='keep').astype('int64')

In [57]:
print(rookie_starter.head(10))

    Season                 Name Team   W   L   G  GS  CG  ShO  QS  ...  \
3     1981  Fernando Valenzuela  LAD  13   7  25  25  11    8  16  ...   
0     1984        Dwight Gooden  NYM  17   9  31  31   7    3  21  ...   
1     1975      John Montefusco  SFG  15   9  34  34  10    4  23  ...   
7     1980          Britt Burns  CHW  14  13  32  32  11    1  21  ...   
2     1995           Hideo Nomo  LAD  13   6  28  28   4    3  18  ...   
20    1984       Orel Hershiser  LAD   9   6  20  20   8    4  16  ...   
14    2013       José Fernández  MIA  12   6  28  28   0    0  20  ...   
10    2003         Brandon Webb  ARI  10   9  28  28   1    1  21  ...   
51    2018       Walker Buehler  LAD   8   4  23  23   0    0  13  ...   
36    1981        Dave Righetti  NYY   8   4  15  15   2    0  13  ...   

    WPA_rank  WPA/LI_rank  K/BB+_rank  K-BB%_rank  Clutch_rank   QS_rank  \
3   1.000000     1.000000    1.000000    1.000000     1.000000  1.000000   
0   1.000000     1.000000    0.91

# Batters

## Rookies

In [58]:
### Batters have a minimum of 300 PA in a Season

batter = pd.read_csv('../data/rookies/fg_bat.csv')
batter.drop(columns=['NameASCII', 'PlayerId'], inplace=True)
batter = batter[~(batter['Season'].isin([1994, 2020]))]

batter = assign_decade(batter)

batter['SB'] = batter['SB'].fillna(0) # fill in stolen bases with 0
batter['CS'] = batter['CS'].fillna(0) # fill in caught stealing with 0

batter['sb_att'] = batter['SB'] + batter['CS'] # stolen base attempts
batter['sb_pct'] = batter['SB'] / batter['sb_att'] # stolen base percentage
batter['WAR/G'] = (batter['WAR']) / batter['G'] # WAR per game

batter['sb_pct'] = batter['sb_pct'].fillna(0)

batter_cols = batter.columns.tolist()
print(f"Columns for Rookie Batters:")
cli.columnize(batter_cols, displaywidth=50)

Columns for Rookie Batters:
Season  PA   BB  AVG   Off   OBP+  WPA/LI   decade
Name    HR   SO  BB/K  wRC+  SLG+  Clutch   sb_att
Team    R    SB  wOBA  BB%+  ISO+  WAR      sb_pct
G       RBI  CS  Def   K%+   WPA   MLBAMID  WAR/G 


In [59]:
# Dictionary for mapping old abbreviations to current ones
team_map = {
    'CAL': 'LAA',  # California Angels to Los Angeles Angels
    'ANA': 'LAA',  # Anaheim Angels to Los Angeles Angels
    'FLA': 'MIA',  # Florida Marlins to Miami Marlins
    'MON': 'WSN',  # Montreal Expos to Washington Nationals
    'TBD': 'TBR'   # Tampa Bay Devil Rays to Tampa Bay Rays
}

# Replace the 'Team' column with updated abbreviations using .replace()
batter['Team'] = batter['Team'].replace(team_map)

In [60]:
nl_teams = ['LAD', 'SDP', 'CIN', 'SFG', 'ATL',
            'NYM', 'MIA', 'MIL', 'CHC', 'STL',
            'COL', 'PIT', 'ARI', 'PHI', 'WSN']
al_teams = ['OAK', 'HOU', 'LAA', 'SEA', 'CLE',
            'BAL', 'NYY', 'BOS', 'KCR', 'MIN',
            'TOR', 'TEX', 'CHW', 'TBR', 'DET']
# add league for the Relievers
batter.loc[batter['Team'].isin(nl_teams), 'league'] = 'NL'
batter.loc[batter['Team'].isin(al_teams), 'league'] = 'AL'

In [61]:
print(f"There are {batter.shape[0]} Batters in the data")

There are 786 Batters in the data


### Rookie of the Month/Year

In [62]:
roy = pd.read_csv('../data/awards/roy_awards.csv')
roy.rename(columns={'player_id' : 'rookie_of_the_year'}, inplace=True)

print(f"Columns for Rookie of the Year data:")
print(roy.columns.tolist())
print("\n")

Columns for Rookie of the Year data:
['award_id', 'season', 'rookie_of_the_year']




In [63]:
batter = pd.merge(batter, roy,
        left_on = 'Season', right_on = 'season',
        how = 'left')

batter.drop(columns='season', inplace=True)

batter.loc[batter['award_id'] == 'ALROY', 'award_id'] = 'AL'
batter.loc[batter['award_id'] == 'NLROY', 'award_id'] = 'NL'
batter.loc[batter['award_id'].isna(), 'award_id'] = batter.loc[batter['award_id'].isna(), 'league']

batter = batter.drop_duplicates(subset=['Season', 'MLBAMID'])

batter.dropna(subset='league', inplace=True)

In [64]:
batter_24 = batter[batter['Season'] == 2024]

batter = batter[batter['Season'] != 2024]

In [65]:
batter['rookie_of_the_year'] = np.where(batter['rookie_of_the_year'] == batter['MLBAMID'], 1, 0)

print(batter.head(10))
print("\n")
print(batter_24.head(10))

    Season               Name Team    G   PA  HR    R  RBI   BB   SO  ...  \
0     2012         Mike Trout  LAA  139  639  30  129   83   67  139  ...   
2     2017        Aaron Judge  NYY  155  678  52  128  114  127  208  ...   
4     1993        Mike Piazza  LAD  149  602  35   81  112   46   86  ...   
6     2001      Albert Pujols  STL  161  676  37  112  130   69   93  ...   
8     1975          Fred Lynn  BOS  145  605  21  103  105   62   90  ...   
10    1997  Nomar Garciaparra  BOS  153  734  30  122   98   35   92  ...   
12    1977      Mitchell Page  OAK  145  592  21   85   75   78   95  ...   
14    2015        Kris Bryant  CHC  151  650  26   87   99   77  199  ...   
16    2001      Ichiro Suzuki  SEA  157  738   8  127   69   30   53  ...   
18    1992       Kenny Lofton  CLE  148  651   5   96   42   68   54  ...   

      Clutch        WAR  MLBAMID  decade  sb_att    sb_pct     WAR/G  league  \
0  -0.321035  10.064204   545361   2010s      54  0.907407  0.072404    

In [66]:
cli.columnize(batter_cols, displaywidth=50)

Season  PA   BB  AVG   Off   OBP+  WPA/LI   decade
Name    HR   SO  BB/K  wRC+  SLG+  Clutch   sb_att
Team    R    SB  wOBA  BB%+  ISO+  WAR      sb_pct
G       RBI  CS  Def   K%+   WPA   MLBAMID  WAR/G 


In [67]:
low_cols = ['K%+']
high_cols = ['wRC+', 'BB%+', 'BB/K', 'OBP+', 'SLG+', 
             'ISO+','WAR', 'WAR/G', 'wOBA', 'Def', 'Off',
             'PA', 'WPA', 'WPA/LI', 'Clutch']

In [68]:
# Rank the low_cols in descending order with percentile-based ranking
batter[[col + '_rank' for col in low_cols]] = batter.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                      na_option='keep',
                                                                                      pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
batter[[col + '_rank' for col in high_cols]] = batter.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                        na_option='keep',
                                                                                        pct=True)

# Select columns that end with '_rank'
rank_columns = batter.filter(regex='_rank$').columns.tolist()

# Calculate the mean rank
batter['mean_rookie_rank'] = batter[rank_columns].mean(axis=1)

# Rank mean_rookie_rank in descending order with percentile-based ranking
batter['rookie_rank'] = batter.groupby('Season')['mean_rookie_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [69]:
# Rank the low_cols in descending order with percentile-based ranking
batter_24[[col + '_rank' for col in low_cols]] = batter_24.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                            na_option='keep',
                                                                                            pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
batter_24[[col + '_rank' for col in high_cols]] = batter_24.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                              na_option='keep',
                                                                                              pct=True)

# Select columns that end with '_rank'
rank_columns = batter_24.filter(regex='_rank$').columns.tolist()

# Calculate the mean rank
batter_24['mean_rookie_rank'] = batter_24[rank_columns].mean(axis=1)

# Rank mean_rookie_rank in descending order with percentile-based ranking
batter_24['rookie_rank'] = batter_24.groupby('Season')['mean_rookie_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [70]:
add_cols = ['rookie_of_the_year', 'mean_rookie_rank', 'rookie_rank']
rank_columns = rank_columns + add_cols

rank_columns.insert(0, 'Name')
rank_columns.insert(1, 'Season')
print(batter.loc[:, rank_columns].sort_values('mean_rookie_rank', ascending=False).head(10))

                Name  Season  K%+_rank  wRC+_rank  BB%+_rank  BB/K_rank  \
137       Tim Raines    1981  0.800000   1.000000   1.000000   1.000000   
28       Alvin Davis    1984  0.666667   0.888889   1.000000   1.000000   
163    Butch Wynegar    1976  0.750000   1.000000   0.750000   1.000000   
0         Mike Trout    2012  0.294118   1.000000   1.000000   0.823529   
58     Jason Heyward    2010  0.458333   0.958333   0.958333   0.958333   
282      Todd Helton    1998  0.909091   0.909091   0.818182   1.000000   
22   Adley Rutschman    2022  0.875000   0.916667   1.000000   0.916667   
14       Kris Bryant    2015  0.172414   0.896552   0.931034   0.758621   
6      Albert Pujols    2001  0.500000   1.000000   1.000000   0.900000   
8          Fred Lynn    1975  0.454545   1.000000   0.909091   0.636364   

     OBP+_rank  SLG+_rank  ISO+_rank  WAR_rank  ...  wOBA_rank  Def_rank  \
137   1.000000   1.000000   1.000000  1.000000  ...   1.000000  0.600000   
28    0.888889   1.000

In [71]:
print(batter_24.loc[:, rank_columns].sort_values('mean_rookie_rank', ascending=False).head(10))

                Name  Season  K%+_rank  wRC+_rank  BB%+_rank  BB/K_rank  \
606  Spencer Horwitz    2024  0.904762   1.000000   0.761905   0.904762   
151     Austin Wells    2024  0.476190   0.857143   0.904762   0.857143   
82   Jackson Merrill    2024  0.809524   0.952381   0.095238   0.190476   
206  Jackson Chourio    2024  0.523810   0.761905   0.428571   0.523810   
253     Wilyer Abreu    2024  0.095238   0.904762   0.619048   0.333333   
254       Joey Ortiz    2024  0.666667   0.666667   1.000000   0.952381   
521    Michael Busch    2024  0.142857   0.714286   0.857143   0.666667   
158    Colton Cowser    2024  0.047619   0.809524   0.666667   0.428571   
259       Masyn Winn    2024  0.857143   0.619048   0.380952   0.619048   
750   Wyatt Langford    2024  0.428571   0.523810   0.571429   0.714286   

     OBP+_rank  SLG+_rank  ISO+_rank  WAR_rank  ...  wOBA_rank  Def_rank  \
606   1.000000   0.904762   0.904762  0.523810  ...   1.000000  0.238095   
151   0.904762   0.761

In [72]:
rookie_batters = batter.copy()
rookie_batters_24 = batter_24.copy()
rookie_batters['pos'] = 'batter'
rookie_batters_24['pos'] = 'batter'

## All MLB Batters

In [73]:
batter = pd.read_csv('../data/mlb/fg_bat.csv')
batter.drop(columns=['NameASCII', 'PlayerId'], inplace=True)
batter = batter[~(batter['Season'].isin([1994, 2020]))]

batter = assign_decade(batter)

batter['SB'] = batter['SB'].fillna(0)
batter['CS'] = batter['CS'].fillna(0)

batter['sb_att'] = batter['SB'] + batter['CS']
batter['sb_pct'] = batter['SB'] / batter['sb_att']
batter['WAR/G'] = (batter['WAR']) / batter['G']

batter['sb_pct'] = batter['sb_pct'].fillna(0)

batter_cols = batter.columns.tolist()
print(f"Columns for Rookie Batters:")
cli.columnize(batter_cols, displaywidth=50)

Columns for Rookie Batters:
Season  PA   BB  AVG   Off   OBP+  WPA/LI   decade
Name    HR   SO  BB/K  wRC+  SLG+  Clutch   sb_att
Team    R    SB  wOBA  BB%+  ISO+  WAR      sb_pct
G       RBI  CS  Def   K%+   WPA   MLBAMID  WAR/G 


In [74]:
cli.columnize(batter.columns.tolist(), displaywidth = 50)

Season  PA   BB  AVG   Off   OBP+  WPA/LI   decade
Name    HR   SO  BB/K  wRC+  SLG+  Clutch   sb_att
Team    R    SB  wOBA  BB%+  ISO+  WAR      sb_pct
G       RBI  CS  Def   K%+   WPA   MLBAMID  WAR/G 


In [75]:
# Dictionary for mapping old abbreviations to current ones
team_map = {
    'CAL': 'LAA',  # California Angels to Los Angeles Angels
    'ANA': 'LAA',  # Anaheim Angels to Los Angeles Angels
    'FLA': 'MIA',  # Florida Marlins to Miami Marlins
    'MON': 'WSN',  # Montreal Expos to Washington Nationals
    'TBD': 'TBR'   # Tampa Bay Devil Rays to Tampa Bay Rays
}

# Replace the 'Team' column with updated abbreviations using .replace()
batter['Team'] = batter['Team'].replace(team_map)

In [76]:
nl_teams = ['LAD', 'SDP', 'CIN', 'SFG', 'ATL',
            'NYM', 'MIA', 'MIL', 'CHC', 'STL',
            'COL', 'PIT', 'ARI', 'PHI', 'WSN']
al_teams = ['OAK', 'HOU', 'LAA', 'SEA', 'CLE',
            'BAL', 'NYY', 'BOS', 'KCR', 'MIN',
            'TOR', 'TEX', 'CHW', 'TBR', 'DET']
# add league for batters
batter.loc[batter['Team'].isin(nl_teams), 'league'] = 'NL'
batter.loc[batter['Team'].isin(al_teams), 'league'] = 'AL'

In [77]:
print(f"There are {batter.shape[0]} Batters in the data")

There are 12481 Batters in the data


In [78]:
batter.dropna(subset='league', inplace=True)

In [79]:
batter = batter.drop_duplicates(subset=['Season', 'MLBAMID'])

batter_24 = batter[batter['Season'] == 2024]

batter = batter[batter['Season'] != 2024]

In [80]:
print(batter.head(10))

   Season              Name Team    G   PA  HR    R  RBI   BB   SO  ...  \
0    2002       Barry Bonds  SFG  143  612  46  117  110  198   47  ...   
1    2001       Barry Bonds  SFG  153  664  73  129  137  177   93  ...   
2    2004       Barry Bonds  SFG  147  617  45  129  101  232   41  ...   
3    2022       Aaron Judge  NYY  157  696  62  133  131  111  175  ...   
4    1975        Joe Morgan  CIN  146  639  17  107   94  132   52  ...   
5    1991    Cal Ripken Jr.  BAL  162  717  34   99  114   53   46  ...   
6    1993       Barry Bonds  SFG  159  674  46  129  123  126   79  ...   
7    2003       Barry Bonds  SFG  130  550  45  111   90  148   58  ...   
8    1990  Rickey Henderson  OAK  136  594  28  119   61   97   60  ...   
9    2018      Mookie Betts  BOS  136  614  32  129   80   81   91  ...   

         WPA     WPA/LI    Clutch        WAR  MLBAMID  decade  sb_att  \
0  10.506586  11.816461 -1.113779  12.691475   111188   2000s      11   
1  11.634218  13.042961 -1.0

In [81]:
print(batter_24.head(10))

      Season              Name Team    G   PA  HR    R  RBI   BB   SO  ...  \
21      2024       Aaron Judge  NYY  144  644  51  110  126  119  156  ...   
22      2024    Bobby Witt Jr.  KCR  147  647  30  120   99   47   98  ...   
144     2024         Juan Soto  NYY  142  650  39  115  100  117  105  ...   
176     2024  Francisco Lindor  NYM  146  665  31  102   85   54  122  ...   
178     2024  Gunnar Henderson  BAL  145  658  36  111   85   72  145  ...   
250     2024     Shohei Ohtani  LAD  143  656  47  116  104   74  146  ...   
392     2024      Jarren Duran  BOS  144  663  21  102   72   50  144  ...   
458     2024   Elly De La Cruz  CIN  145  631  23   96   65   65  194  ...   
792     2024       Ketel Marte  ARI  120  517  31   83   85   53   94  ...   
1013    2024      José Ramírez  CLE  143  621  34  103  106   51   76  ...   

           WPA    WPA/LI    Clutch       WAR  MLBAMID  decade  sb_att  \
21    4.933473  8.232481 -2.868269  9.621307   592450   2020s       

In [82]:
# Rank the low_cols in descending order with percentile-based ranking
batter[[col + '_rank' for col in low_cols]] = batter.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                      na_option='keep',
                                                                                      pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
batter[[col + '_rank' for col in high_cols]] = batter.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                        na_option='keep',
                                                                                        pct=True)

# Select columns that end with '_rank'
rank_columns = batter.filter(regex='_rank$').columns.tolist()

# Calculate the mean rank for MLB data
batter['mean_mlb_rank'] = batter[rank_columns].mean(axis=1)

# Rank mean_mlb_rank in descending order with percentile-based ranking
batter['mlb_rank'] = batter.groupby('Season')['mean_mlb_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [83]:
# Rank the low_cols in descending order with percentile-based ranking
batter_24[[col + '_rank' for col in low_cols]] = batter_24.groupby('Season')[low_cols].rank(ascending=False, 
                                                                                            na_option='keep',
                                                                                            pct=True)

# Rank the high_cols in ascending order with percentile-based ranking
batter_24[[col + '_rank' for col in high_cols]] = batter_24.groupby('Season')[high_cols].rank(ascending=True, 
                                                                                              na_option='keep',
                                                                                              pct=True)

# Select columns that end with '_rank'
rank_columns = batter_24.filter(regex='_rank$').columns.tolist()

# Calculate the mean rank for MLB data
batter_24['mean_mlb_rank'] = batter_24[rank_columns].mean(axis=1)

# Rank mean_mlb_rank in descending order with percentile-based ranking
batter_24['mlb_rank'] = batter_24.groupby('Season')['mean_mlb_rank'].rank(ascending=False, na_option='keep').astype('int64')

In [84]:
add_cols = ['mean_mlb_rank', 'mlb_rank']
rank_columns = rank_columns + add_cols

rank_columns.insert(0, 'Name')
rank_columns.insert(1, 'Season')
print(batter.loc[:, rank_columns].sort_values('mean_mlb_rank', ascending=False).head(10))

              Name  Season  K%+_rank  wRC+_rank  BB%+_rank  BB/K_rank  \
77     Todd Helton    2000  0.945148   0.974684   0.890295   0.995781   
103   Alex Bregman    2018  0.940000   0.980000   0.928000   0.992000   
93   Albert Pujols    2006  0.966805   0.995851   0.917012   0.995851   
122    Barry Bonds    1991  0.693694   0.990991   0.990991   0.963964   
128    Barry Bonds    1995  0.700935   0.967290   0.990654   0.962617   
135  Albert Pujols    2007  0.939850   0.969925   0.943609   0.996241   
71   Albert Pujols    2009  0.956522   1.000000   0.976285   1.000000   
2      Barry Bonds    2004  0.980392   1.000000   1.000000   1.000000   
127     Wade Boggs    1986  0.969432   0.991266   0.978166   0.991266   
132     Wade Boggs    1983  0.957082   1.000000   0.935622   1.000000   

     OBP+_rank  SLG+_rank  ISO+_rank  WAR_rank  WAR/G_rank  wOBA_rank  \
77    0.987342   0.991561   0.974684  0.991561    0.974684   0.991561   
103   0.972000   0.940000   0.900000  0.992000    

In [85]:
print(batter_24.loc[:, rank_columns].sort_values('mean_mlb_rank', ascending=False).head(10))

                  Name  Season  K%+_rank  wRC+_rank  BB%+_rank  BB/K_rank  \
144          Juan Soto    2024  0.817427   0.995851   0.995851   0.995851   
22      Bobby Witt Jr.    2024  0.850622   0.979253   0.435685   0.738589   
792        Ketel Marte    2024  0.709544   0.946058   0.767635   0.858921   
176   Francisco Lindor    2024  0.701245   0.887967   0.526971   0.692946   
1564      Corey Seager    2024  0.713693   0.912863   0.746888   0.838174   
178   Gunnar Henderson    2024  0.431535   0.966805   0.850622   0.742739   
1483      Mookie Betts    2024  0.979253   0.958506   0.970954   1.000000   
2091  Jurickson Profar    2024  0.858921   0.917012   0.904564   0.987552   
21         Aaron Judge    2024  0.348548   1.000000   1.000000   0.983402   
1013      José Ramírez    2024  0.970954   0.867220   0.572614   0.946058   

      OBP+_rank  SLG+_rank  ISO+_rank  WAR_rank  WAR/G_rank  wOBA_rank  \
144    0.995851   0.987552   0.991701  0.991701    0.991701   0.995851   
22  

In [86]:
rookie_batters = pd.merge(rookie_batters, batter[['Season', 'MLBAMID', 'mean_mlb_rank', 'mlb_rank']],
                           on = ['Season', 'MLBAMID'],
                           how='left'
                          ).sort_values('mean_rookie_rank', ascending=False)
rookie_batters_24 = pd.merge(rookie_batters_24, batter_24[['Season', 'MLBAMID', 'mean_mlb_rank', 'mlb_rank']],
                           on = ['Season', 'MLBAMID'],
                           how='left'
                          ).sort_values('mean_rookie_rank', ascending=False)

rookie_batters = rookie_batters.drop_duplicates(subset=['Season', 'MLBAMID'])
rookie_batters_24 = rookie_batters_24.drop_duplicates(subset=['Season', 'MLBAMID'])

print(rookie_batters.head(10))

     Season             Name Team    G   PA  HR    R  RBI  BB   SO  ...  \
68     1981       Tim Raines  WSN   88  363   5   61   37  45   31  ...   
14     1984      Alvin Davis  SEA  152  678  27   80  116  97   78  ...   
80     1976    Butch Wynegar  MIN  149  622  10   58   69  79   63  ...   
0      2012       Mike Trout  LAA  139  639  30  129   83  67  139  ...   
29     2010    Jason Heyward  ATL  142  623  18   83   72  91  128  ...   
137    1998      Todd Helton  COL  152  595  25   78   97  53   54  ...   
11     2022  Adley Rutschman  BAL  113  470  13   70   42  65   86  ...   
7      2015      Kris Bryant  CHC  151  650  26   87   99  77  199  ...   
3      2001    Albert Pujols  STL  161  676  37  112  130  69   93  ...   
4      1975        Fred Lynn  BOS  145  605  21  103  105  62   90  ...   

     Off_rank   PA_rank  WPA_rank  WPA/LI_rank  Clutch_rank  mean_rookie_rank  \
68   1.000000  0.600000  1.000000     1.000000     0.800000          0.925000   
14   1.00000

In [87]:
print(rookie_batters_24.head(10))

    Season             Name Team    G   PA  HR   R  RBI  BB   SO  ...  \
10    2024  Spencer Horwitz  TOR   83  320  12  42   35  34   56  ...   
1     2024     Austin Wells  NYY  102  362  13  39   50  41   75  ...   
0     2024  Jackson Merrill  SDP  142  538  23  72   83  25   96  ...   
3     2024  Jackson Chourio  MIL  131  501  19  70   69  35  104  ...   
4     2024     Wilyer Abreu  BOS  116  392  15  55   55  34  113  ...   
5     2024       Joey Ortiz  MIL  125  448  10  49   49  54   88  ...   
9     2024    Michael Busch  CHC  137  516  17  68   56  58  149  ...   
2     2024    Colton Cowser  BAL  139  507  20  70   62  49  153  ...   
6     2024       Masyn Winn  STL  135  567  12  70   49  38  101  ...   
14    2024   Wyatt Langford  TEX  118  487  10  58   63  41  102  ...   

    Off_rank   PA_rank  WPA_rank  WPA/LI_rank  Clutch_rank  mean_rookie_rank  \
10  0.809524  0.095238  0.952381     0.952381     1.000000          0.791667   
1   0.714286  0.190476  0.809524    

In [88]:
print(f"There are {rookie_batters.shape[0]} Batters in the data")

There are 748 Batters in the data


## Rookie of the Year Voting

In [89]:
voting = pd.read_csv('../data/awards/roy_voting.csv')
voting.head(10)

,awardID,yearID,lgID,playerID,pointsWon,pointsMax,votesFirst,unanimous,rookie_of_the_year
0,Rookie of the Year,1949,NL,newcodo01,21,24,21,0,1
1,Rookie of the Year,1949,AL,sievero01,10,24,10,0,1
2,Rookie of the Year,1950,AL,dropowa01,15,24,15,0,1
3,Rookie of the Year,1950,NL,jethrsa01,11,24,11,0,1
4,Rookie of the Year,1951,NL,mayswi01,18,24,18,0,1
5,Rookie of the Year,1951,AL,mcdougi01,13,24,13,0,1
6,Rookie of the Year,1952,NL,blackjo02,19,24,19,0,1
7,Rookie of the Year,1952,AL,byrdha01,9,24,9,0,1
8,Rookie of the Year,1953,AL,kuennha01,23,24,23,0,1
9,Rookie of the Year,1953,NL,gilliji01,11,24,11,0,1


In [90]:
print("Columns for Rookie of the Year Voting: \n")
cli.columnize(voting.columns.tolist(), displaywidth=50)

Columns for Rookie of the Year Voting: 

awardID  playerID   votesFirst        
yearID   pointsWon  unanimous         
lgID     pointsMax  rookie_of_the_year


## Player IDs

In [91]:
player_ids = pd.read_csv('../data/cleaned_player_ids.csv', low_memory=False,
                        usecols=['key_mlbam', 'key_bbref', 'key_fangraphs', 'key_retro'])

In [92]:
print("Columns in the Player ID's Data: \n")
cli.columnize(player_ids.columns.tolist(), displaywidth=50)

Columns in the Player ID's Data: 

key_mlbam  key_retro  key_bbref  key_fangraphs


In [93]:
def add_ids(df, ids = player_ids):
    df = pd.merge(df, ids[['key_mlbam', 'key_bbref']].drop_duplicates(subset=['key_mlbam']),
                    left_on = 'MLBAMID', right_on = 'key_mlbam',
                    how = 'inner'
                   ).drop(columns='key_mlbam')
    
    return df

rookie_relief = add_ids(df=rookie_relief)
rookie_relief_24 = add_ids(df=rookie_relief_24)

rookie_starter = add_ids(df=rookie_starter)
rookie_starter_24 = add_ids(df=rookie_starter_24)

In [94]:
rookie_batters = add_ids(df=rookie_batters)

## Add Voting data to `pitchers` and `batters`

In [95]:
def add_votes(df, votes=voting):
    df = pd.merge(df, votes.drop(columns='rookie_of_the_year').drop_duplicates(subset=['yearID', 'playerID'], keep='first'),
                  left_on='key_bbref', right_on='playerID', 
                  how='left').drop(columns='playerID').drop_duplicates(subset='MLBAMID')

    df['awardID'] = 'Rookie of the Year'
    df['yearID'] = df['yearID'].fillna(df['Season'])
    df['lgID'] = df['lgID'].fillna(df['league'])
    df['vote_share'] = df['pointsWon'] / df['pointsMax']

    for col in ['pointsWon', 'pointsMax', 'votesFirst', 'vote_share', 'unanimous']:
        df[col] = df[col].fillna(0)

    df['vote_getter'] = np.where(df['pointsMax'] > 0, 1, 0)

    return df

rookie_relief = add_votes(df = rookie_relief)
rookie_relief_24 = add_votes(df = rookie_relief_24)
rookie_starter = add_votes(df = rookie_starter)
rookie_starter_24 = add_votes(df = rookie_starter_24)

print(rookie_relief.head(10))

   Season               Name Team   G     IP   SO       ERA       ERA-  \
0    2014    Dellin Betances  NYY  70   90.0  135  1.400000  36.423515   
1    2006  Jonathan Papelbon  BOS  59   68.1   75  0.921952  19.523037   
2    1995      Troy Percival  LAA  62   74.0   94  1.945947  40.697834   
3    2006      Takashi Saito  LAD  72   78.1  107  2.068086  46.986878   
4    1986      Mark Eichhorn  TOR  69  157.0  166  1.719745  40.860984   
5    2022        Jhoan Duran  MIN  57   67.2   89  1.862069  48.296279   
6    1980       Doug Corbett  MIN  73  136.1   89  1.980440  47.446845   
7    2011       Greg Holland  KCR  46   60.0   74  1.800000  43.297549   
8    2007       Joakim Soria  KCR  62   69.0   75  2.478262  54.023029   
9    2009      Andrew Bailey  OAK  68   83.1   91  1.836000  42.511891   

        FIP       FIP-  ...  key_bbref             awardID  yearID  lgID  \
0  1.643363  42.231888  ...  betande01  Rookie of the Year  2014.0    AL   
1  2.136991  48.966961  ...  pape

In [96]:
rookie_batters = rookie_batters.dropna(subset='league')
rookie_batters = add_votes(df = rookie_batters)
print(rookie_batters.head(10))

   Season             Name Team    G   PA  HR    R  RBI  BB   SO  ...  \
0    1981       Tim Raines  WSN   88  363   5   61   37  45   31  ...   
1    1984      Alvin Davis  SEA  152  678  27   80  116  97   78  ...   
2    1976    Butch Wynegar  MIN  149  622  10   58   69  79   63  ...   
3    2012       Mike Trout  LAA  139  639  30  129   83  67  139  ...   
4    2010    Jason Heyward  ATL  142  623  18   83   72  91  128  ...   
5    1998      Todd Helton  COL  152  595  25   78   97  53   54  ...   
6    2022  Adley Rutschman  BAL  113  470  13   70   42  65   86  ...   
7    2015      Kris Bryant  CHC  151  650  26   87   99  77  199  ...   
8    2001    Albert Pujols  STL  161  676  37  112  130  69   93  ...   
9    1975        Fred Lynn  BOS  145  605  21  103  105  62   90  ...   

   key_bbref             awardID  yearID  lgID  pointsWon  pointsMax  \
0  raineti01  Rookie of the Year  1981.0    NL       85.0      120.0   
1  davisal01  Rookie of the Year  1984.0    AL      

In [97]:
rookie_relief['rookie_of_the_year'].value_counts()

rookie_of_the_year
0    698
1      9
Name: count, dtype: int64

In [98]:
rookie_starter['rookie_of_the_year'].value_counts()

rookie_of_the_year
0    447
1     15
Name: count, dtype: int64

In [99]:
rookie_batters['rookie_of_the_year'].value_counts()

rookie_of_the_year
0    710
1     38
Name: count, dtype: int64

In [100]:
def calc_WPA_rate(data, per_game=False):
    if per_game:
        data['WPA/G'] = data['WPA'] / data['G']
    else:
        data['WPA/IP'] = data['WPA'] / data['IP']

    return data

rookie_relief = calc_WPA_rate(data = rookie_relief, per_game=False)
rookie_starter = calc_WPA_rate(data = rookie_starter, per_game=False)
rookie_batters = calc_WPA_rate(data = rookie_batters, per_game=True)

rookie_relief_24 = calc_WPA_rate(data = rookie_relief_24, per_game=False)
rookie_starter_24 = calc_WPA_rate(data = rookie_starter_24, per_game=False)
rookie_batters_24 = calc_WPA_rate(data = rookie_batters_24, per_game=True)

In [101]:
rookie_relief.to_csv('../data/cleaned/rookie_relief.csv', index=False)
rookie_starter.to_csv('../data/cleaned/rookie_starter.csv', index=False)
rookie_batters.to_csv('../data/cleaned/rookie_batters.csv', index=False)

rookie_relief_24.to_csv('../data/cleaned/rookie_relief_24.csv', index=False)
rookie_starter_24.to_csv('../data/cleaned/rookie_starter_24.csv', index=False)
rookie_batters_24.to_csv('../data/cleaned/rookie_batters_24.csv', index=False)

In [102]:
end_time = time.time()
print(f"Total processing time = {(end_time - start_time):.2f} seconds")

Total processing time = 0.97 seconds
